# TODO: Title
**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of the TODO's and/or use more than one cell to complete all the tasks.

In [1]:
%%capture
!pip install smdebug torch torchvision tqdm

In [2]:
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

## Data Preparation
**TODO:** Run the cell below to download the data.

The cell below creates a folder called `train_data`, downloads training data and arranges it in subfolders. Each of these subfolders contain images where the number of objects is equal to the name of the folder. For instance, all images in folder `1` has images with 1 object in them. Images are not divided into training, testing or validation sets. If you feel like the number of samples are not enough, you can always download more data (instructions for that can be found [here](https://registry.opendata.aws/amazon-bin-imagery/)). However, we are not acessing you on the accuracy of your final trained model, but how you create your machine learning engineering pipeline.

In [102]:
# test train test split
from sklearn.model_selection import train_test_split
import json

# open and read json
with open('file_list.json', 'r') as f:
    lines = f.readlines()

print(len(lines[0]))

import pandas as pd
print(type(json.loads(lines[0])))

res = json.loads(lines[0])
# res
# res.items()
# bin_list = []
# bin_list.append(res)
# bin_list[0]

# df = pd.DataFrame(data = ((k, item) for k, v in res.items() for item in v), columns = ['label', 'data'])
# df.head()
# df = pd.DataFrame(bin_list)
# df

# type(lines[0]) 

# train, test = train_test_split(df, test_size=0.2)
# print(test)

l = [(k,bin) for k,v in res.items() for bin in v]
import random
random.shuffle(l)
train_size = int(len(l)*0.8)
train, test = l[: train_size], l[train_size:]
len(train), len(test)

295763
<class 'dict'>


(8352, 2089)

In [134]:
test

[('2', 'data/metadata/03118.json'),
 ('2', 'data/metadata/02563.json'),
 ('3', 'data/metadata/07574.json'),
 ('2', 'data/metadata/05159.json'),
 ('5', 'data/metadata/04176.json'),
 ('3', 'data/metadata/100306.json'),
 ('1', 'data/metadata/04850.json'),
 ('2', 'data/metadata/102034.json'),
 ('2', 'data/metadata/03414.json'),
 ('4', 'data/metadata/08896.json'),
 ('4', 'data/metadata/103650.json'),
 ('5', 'data/metadata/104545.json'),
 ('5', 'data/metadata/03960.json'),
 ('3', 'data/metadata/04246.json'),
 ('4', 'data/metadata/01290.json'),
 ('1', 'data/metadata/06008.json'),
 ('5', 'data/metadata/09649.json'),
 ('2', 'data/metadata/103003.json'),
 ('2', 'data/metadata/10407.json'),
 ('2', 'data/metadata/103061.json'),
 ('2', 'data/metadata/08118.json'),
 ('3', 'data/metadata/08481.json'),
 ('1', 'data/metadata/07763.json'),
 ('5', 'data/metadata/102112.json'),
 ('3', 'data/metadata/03865.json'),
 ('4', 'data/metadata/09640.json'),
 ('3', 'data/metadata/09516.json'),
 ('2', 'data/metadata

In [133]:
# test
test_json = {
    "1": [],
    "2": [],
    "3": [],
    "4": [],
    "5": []
}

for k, i in test:
    test_json[k].append(i)
    
sum([len(x) for x in test_json.values()])
type(test_json)


dict

In [135]:
with open("test.json", "w") as fp:
        json.dump(test_json, fp)

In [136]:
import json
import random

def random_split(filename: str):
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    # load json
    res = json.loads(lines[0]) 

    # split file to train, test
    l = [(k,bin) for k,v in res.items() for bin in v]

    random.shuffle(l)
    train_size = int(len(l)*0.8)
    train, test = l[: train_size], l[train_size:]
    return train, test

In [145]:
# save file to json folder and maintain struture of original file

def save_json(file):
    file_json = {
    "1": [],
    "2": [],
    "3": [],
    "4": [],
    "5": []
    }

    if file == train_list:
        filename = 'train_list.json'
    else:
        filename = 'test_list.json'
    # filename = str(file) + ".json"  

    for k, i in file:
        file_json[k].append(i)
    
    # sum([len(x) for x in test_json.values()])
    with open(filename, "w") as fp:
        json.dump(file_json, fp)
    


In [146]:
train_list, test_list = random_split("file_list.json")


In [147]:
save_json(train_list)
save_json(test_list)

In [61]:
# plot data distribution

In [ ]:
# modify file to accept input filename

import os
import json
import boto3
from tqdm import tqdm

def download_and_arrange_data():
    s3_client = boto3.client('s3')

    with open('file_list.json', 'r') as f:
        d=json.load(f)

    for k, v in d.items():
        print(f"Downloading Images with {k} objects")
        directory=os.path.join('train_data', k)
        if not os.path.exists(directory):
            os.makedirs(directory)
        for file_path in tqdm(v):
            file_name=os.path.basename(file_path).split('.')[0]+'.jpg'
            s3_client.download_file('aft-vbi-pds', os.path.join('bin-images', file_name),
                             os.path.join(directory, file_name))

download_and_arrange_data()

## Dataset
**TODO:** Explain what dataset you are using for this project. Give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understanding of it. You can find more information about the data [here](https://registry.opendata.aws/amazon-bin-imagery/).

In [ ]:
#TODO: Perform any data cleaning or data preprocessing

In [ ]:
#TODO: Upload the data to AWS S3

## Model Training
**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important. 

**Note:** You will need to use the `train.py` script to train your model.

In [ ]:
#TODO: Declare your model training hyperparameter.
#NOTE: You do not need to do hyperparameter tuning. You can use fixed hyperparameter values

In [ ]:
#TODO: Create your training estimator

In [ ]:
# TODO: Fit your estimator

## Standout Suggestions
You do not need to perform the tasks below to finish your project. However, you can attempt these tasks to turn your project into a more advanced portfolio piece.

### Hyperparameter Tuning
**TODO:** Here you can perform hyperparameter tuning to increase the performance of your model. You are encouraged to 
- tune as many hyperparameters as you can to get the best performance from your model
- explain why you chose to tune those particular hyperparameters and the ranges.


In [ ]:
#TODO: Create your hyperparameter search space

In [ ]:
#TODO: Create your training estimator

In [ ]:
# TODO: Fit your estimator

In [ ]:
# TODO: Find the best hyperparameters

### Model Profiling and Debugging
**TODO:** Use model debugging and profiling to better monitor and debug your model training job.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

### Model Deploying and Querying
**TODO:** Can you deploy your model to an endpoint and then query that endpoint to get a result?

In [ ]:
# TODO: Deploy your model to an endpoint

In [ ]:
# TODO: Run an prediction on the endpoint

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done

### Cheaper Training and Cost Analysis
**TODO:** Can you perform a cost analysis of your system and then use spot instances to lessen your model training cost?

In [ ]:
# TODO: Cost Analysis

In [ ]:
# TODO: Train your model using a spot instance

### Multi-Instance Training
**TODO:** Can you train your model on multiple instances?

In [ ]:
# TODO: Train your model on Multiple Instances